# ImageNet Baseline - Simple ResNet

This notebook implements a simple baseline on ImageNet using a ResNet architecture.


In [1]:
# Global preparations
import sys
import os
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import numpy as np

sys.path.insert(0, os.path.expanduser('~/spiky'))
device = 'cuda:5'
random_seed = 1
torch.manual_seed(random_seed)
np.random.seed(random_seed)
torch.backends.cudnn.enabled = True
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")


PyTorch version: 2.1.2+cu121
Device: cuda:5


## Data Loading

ImageNet dataset loading with standard augmentations for training and validation.


In [2]:
# ImageNet data paths (adjust these to your ImageNet location)
imagenet_train_path = '/path/to/imagenet/train'  # Update this path
imagenet_val_path = '/path/to/imagenet/val'      # Update this path

# Standard ImageNet normalization
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Training transforms with augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    normalize,
])

# Validation transforms (no augmentation)
val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize,
])

# Note: You need to have ImageNet dataset downloaded
# For demonstration, we'll use ImageNet-like dataset or CIFAR-100 as a proxy
# Uncomment and update paths when you have ImageNet available
"""
train_dataset = torchvision.datasets.ImageFolder(
    root=imagenet_train_path,
    transform=train_transform
)

val_dataset = torchvision.datasets.ImageFolder(
    root=imagenet_val_path,
    transform=val_transform
)
"""

# Using CIFAR-100 as a smaller proxy for testing (same structure as ImageNet)
# Replace with ImageNet when available
print("Using CIFAR-100 as a proxy dataset. Replace with ImageNet for full experiments.")

train_transform_cifar = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

val_transform_cifar = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])

train_dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=True,
    download=True,
    transform=train_transform_cifar
)

val_dataset = torchvision.datasets.CIFAR100(
    root='./data',
    train=False,
    download=True,
    transform=val_transform_cifar
)

batch_size = 512
num_workers = 4

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")
print(f"Number of classes: {len(train_dataset.classes)}")


Using CIFAR-100 as a proxy dataset. Replace with ImageNet for full experiments.
Files already downloaded and verified
Files already downloaded and verified
Train dataset size: 50000
Val dataset size: 10000
Number of classes: 100


## Baseline Model

We'll use ResNet18 as a simple baseline. This can be easily replaced with ResNet50 or other architectures.


In [4]:
# Load pre-trained ResNet18 and modify for our dataset
# For CIFAR-100, we'll modify the first conv layer to handle 32x32 images
# For ImageNet (224x224), use standard ResNet18

num_classes = len(train_dataset.classes)

if hasattr(train_dataset, 'classes'):
    # CIFAR-100 or similar small dataset
    model = torchvision.models.resnet18(weights=None)  # Start from scratch
    # Modify first conv layer for smaller input
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()  # Remove maxpool for smaller images
    model.fc = nn.Linear(model.fc.in_features, num_classes)
else:
    # ImageNet
    model = torchvision.models.resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model: ResNet18")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Number of classes: {num_classes}")


Model: ResNet18
Total parameters: 11,220,132
Trainable parameters: 11,220,132
Number of classes: 100


## Training Setup

Define loss function, optimizer, and learning rate scheduler.


In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=1e-4
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=30,
    gamma=0.1
)

print("Optimizer: SGD with momentum 0.9, weight decay 1e-4")
print("Initial learning rate: 0.1")
print("Scheduler: StepLR (reduce by 0.1 every 30 epochs)")


Optimizer: SGD with momentum 0.9, weight decay 1e-4
Initial learning rate: 0.1
Scheduler: StepLR (reduce by 0.1 every 30 epochs)


## Training and Evaluation Functions


In [ ]:
def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, leave=False)
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        pbar.set_description(f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%')
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, leave=False)
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_description(f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%')
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


## Training Loop


In [ ]:
num_epochs = 100  # Adjust as needed
train_losses = []
train_accs = []
val_losses = []
val_accs = []

print(f"Starting training for {num_epochs} epochs...")
print(f"Batch size: {batch_size}")
print(f"Device: {device}\n")

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Learning rate: {scheduler.get_last_lr()[0]:.6f}')
    
    # Train
    train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validate
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Update learning rate
    scheduler.step()
    
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print('-' * 60)


## Plot Results


In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
ax1.plot(train_losses, label='Train Loss', color='blue')
ax1.plot(val_losses, label='Val Loss', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training and Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy plot
ax2.plot(train_accs, label='Train Acc', color='blue')
ax2.plot(val_accs, label='Val Acc', color='red')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"Best validation accuracy: {max(val_accs):.2f}% at epoch {val_accs.index(max(val_accs))+1}")


## LUTNet Adaptation to CIFAR-100

Adapting the LUTNet architecture from MNIST to CIFAR-100 classification.


In [218]:
# Import LUTNet components
import sys
import os
sys.path.insert(0, os.path.expanduser('~/spiky'))

from spiky.lut.LUTLayer import (
    Conv2DLUTLayer, LUTSharedContext, SynapseMeta, GradientPolicy, GradientType
)
from spiky.util.torch_utils import make_lr_getter

# Setup LUTNet shared context and metadata
summation_dtype = torch.float32

synapse_meta = SynapseMeta(
    min_weight=-1.0,
    max_weight=1.0,
    initial_weight=0.0,
    initial_noise_level=0.0
)

shared_lut_ctx = LUTSharedContext()
shared_lut_ctx.to_device(device)
g_policy = GradientPolicy(GradientType.Dense, normalized=False)

print("LUTNet components initialized")


LUTNet components initialized


In [243]:
# LUTNet adapted for CIFAR-100 using Conv2DLUTLayer (standard convolutional, no skip connections)
class LutNetCIFAR100(nn.Module):
    def __init__(self, device, num_classes=100):
        super().__init__()
        # CIFAR-100: 32x32 RGB images (3 channels)
        self.filter_lut_1 = Conv2DLUTLayer(
            input_shape=(32, 32 * 3),  # (32, 96) - concatenate 3 channels along width
            n_anchors_per_detector=8,
            detectors_shape=(6, 6),
            output_kernel_shape=(6, 6),
            receptive_field_shape=(5, 5 * 3),
            receptive_field_stride_shape=(2, 2 * 3),
            lut_receptive_field_shape=(6, 6),
            lut_receptive_field_stride_shape=(6, 6),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )
        
        self.filter_lut_2 = Conv2DLUTLayer(
            input_shape=(84, 84),
            n_anchors_per_detector=8,
            detectors_shape=(6, 6),
            output_kernel_shape=(6, 6),
            receptive_field_shape=(5 * 6, 5 * 6),
            receptive_field_stride_shape=(2 * 6, 2 * 6),
            lut_receptive_field_shape=(6, 6),
            lut_receptive_field_stride_shape=(6, 6),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )
        
        self.filter_lut_3 = Conv2DLUTLayer(
            input_shape=(30, 30),
            n_anchors_per_detector=8,
            detectors_shape=(6, 6),
            output_kernel_shape=(6, 6),
            receptive_field_shape=(5 * 6, 5 * 6),
            receptive_field_stride_shape=(2 * 6, 2 * 6),
            lut_receptive_field_shape=(6, 6),
            lut_receptive_field_stride_shape=(6, 6),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )
        
        l = []
        for i in range(6):
            l.append(
                LUTLayer(
                    n_inputs=36,
                    n_anchors_per_detector=10,
                    n_detectors=32,
                    n_outputs=36,
                    synapse_meta=synapse_meta,
                    weights_gradient_policy=g_policy,
                    shared_context=shared_lut_ctx,
                    summation_dtype=summation_dtype,
                    random_seed=random_seed,
                    device=device
                )
            )
        self.c_luts = nn.ModuleList(l)
        self.final_lut = LUTLayer(
            n_inputs=36,
            n_anchors_per_detector=10,
            n_detectors=32,
            n_outputs=num_classes,
            synapse_meta=synapse_meta,
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            device=device
        )

        # self.final_linear = nn.Linear(900, num_classes)

    def forward(self, x):
        B = x.shape[0]
        # CIFAR-100 input: (B, 3, 32, 32)
        # Reshape to (B, 1, 32, 96) by concatenating channels along width dimension
        if x.shape[1] == 3:
            # Reshape: (B, 3, 32, 32) -> (B, 32, 96) -> (B, 1, 32, 96)
            x = x.permute(0, 2, 1, 3).contiguous()  # (B, 3, 32, 32) -> (B, 32, 3, 32)
            x = x.view(B, 32, 32 * 3)  # (B, 32, 96)
            x = x.unsqueeze(1)  # (B, 1, 32, 96)
        else:
            # Already in correct format
            if len(x.shape) == 3:
                x = x.unsqueeze(1)
        
        # First convolutional LUT layer
        x = self.filter_lut_1(x)
        x = self.filter_lut_2(x)
        x = self.filter_lut_3(x).reshape(B, 1, 36)
        
#        return self.final_linear(x)
        
        # Intermediate convolutional LUT layers (no skip connections)
        for c_lut in self.c_luts:
            x = x + c_lut(x)  # No skip connection
        
        # Final output layer
        x = self.final_lut(x)
        
        # Reshape to (B, num_classes)
        return x.reshape(B, -1)
    
    def setup_external_learning_rate_hook(self, optimizer):
        lr_getter = make_lr_getter(optimizer)
        self.filter_lut_1.set_external_learning_rate_hook(lr_getter)
        self.filter_lut_2.set_external_learning_rate_hook(lr_getter)
        for c_lut in self.c_luts:
            c_lut.set_external_learning_rate_hook(lr_getter)
        self.final_lut.set_external_learning_rate_hook(lr_getter)


In [244]:
# Create LUTNet model
lut_net = LutNetCIFAR100(device, num_classes=len(train_dataset.classes))
lut_net = lut_net.to(device)

# Count parameters
lut_total_params = sum(p.numel() for p in lut_net.parameters())
lut_trainable_params = sum(p.numel() for p in lut_net.parameters() if p.requires_grad)

print(f"LUTNet Model: {lut_net}")
print(f"Total parameters: {lut_total_params:,}")
print(f"Trainable parameters: {lut_trainable_params:,}")
print(f"Number of classes: {len(train_dataset.classes)}")


LUTNet Model: LutNetCIFAR100(
  (filter_lut_1): Conv2DLUTLayer(input_shape=(32, 96), output_shape=(84, 84), detectors_shape=(6, 6), n_anchors_per_detector=8)
  (filter_lut_2): Conv2DLUTLayer(input_shape=(84, 84), output_shape=(30, 30), detectors_shape=(6, 6), n_anchors_per_detector=8)
  (filter_lut_3): Conv2DLUTLayer(input_shape=(30, 30), output_shape=(6, 6), detectors_shape=(6, 6), n_anchors_per_detector=8)
  (c_luts): ModuleList(
    (0-5): 6 x LUTLayer(36 inputs, 32 detectors, 36 outputs, 10 anchors per detector)
  )
  (final_lut): LUTLayer(36 inputs, 32 detectors, 100 outputs, 10 anchors per detector)
)
Total parameters: 92,192,762
Trainable parameters: 92,192,762
Number of classes: 100


In [250]:
# Setup optimizer and scheduler for LUTNet
from torch.optim.lr_scheduler import LambdaLR

def lr_func(t):
    return min(
        1.0 / (1 + t)**0.5,
        (t / 4000.0) / 4000.0**0.5
    )

#lr=0.001
lut_optimizer = torch.optim.SGD(
    lut_net.parameters(),
    lr=0.1,
    momentum=0.9,
    weight_decay=1e-4
)
#lut_optimizer = torch.optim.SGD(lut_net.parameters(), lr=0.001)
#lut_optimizer = torch.optim.Adam(lut_net.parameters(), lr=lr)
#lut_sched = None
#lut_sched = LambdaLR(lut_optimizer, lr_lambda=lr_func)
#lut_net.setup_external_learning_rate_hook(lut_optimizer)
lut_sched = torch.optim.lr_scheduler.StepLR(
    lut_optimizer,
    step_size=30,
    gamma=0.9
)

In [246]:
# Training functions for LUTNet
def train_one_epoch_lut(model, train_loader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(train_loader, leave=False)
    for inputs, targets in pbar:
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
#         if scheduler is not None:
#             for _ in range(inputs.shape[0]):
#                 scheduler.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        pbar.set_description(
            f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%, '
            f'lr: {scheduler.get_last_lr()[0] if scheduler is not None else lr:.4f}'
        )
    
    scheduler.step()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


def evaluate_lut(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        pbar = tqdm(val_loader, leave=False)
        for inputs, targets in pbar:
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            pbar.set_description(f'Loss: {loss.item():.4f}, Acc: {100.*correct/total:.2f}%')
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    return epoch_loss, epoch_acc


## LUTNet Training Loop


In [247]:
# LUTNet training
lut_criterion = nn.CrossEntropyLoss()
# def loss_func(o, t):
#     return F.cross_entropy(
#         o, t,
#         reduction='none'
#     ).sum()
lut_train_losses = []
lut_train_accs = []
lut_val_losses = []
lut_val_accs = []

In [ ]:
lut_num_epochs = 100  # Adjust as needed

print(f"Starting LUTNet training for {lut_num_epochs} epochs...")
print(f"Batch size: {batch_size}")
print(f"Device: {device}\n")

for epoch in range(lut_num_epochs):
    print(f'LUTNet Epoch {epoch+1}/{lut_num_epochs}')
    print(f'Learning rate: {lut_sched.get_last_lr()[0] if lut_sched is not None else lr:.6f}')
    
    # Train
    train_loss, train_acc = train_one_epoch_lut(
        lut_net, train_loader, lut_criterion, lut_optimizer, lut_sched, device
    )
    lut_train_losses.append(train_loss)
    lut_train_accs.append(train_acc)
    
    # Validate
    val_loss, val_acc = evaluate_lut(lut_net, val_loader, lut_criterion, device)
    lut_val_losses.append(val_loss)
    lut_val_accs.append(val_acc)
    
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    print('-' * 60)


Starting LUTNet training for 100 epochs...
Batch size: 512
Device: cuda:5

LUTNet Epoch 1/100
Learning rate: 0.053144


  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Train Loss: 3.0334, Train Acc: 26.09%
Val Loss: 3.2043, Val Acc: 23.19%
------------------------------------------------------------
LUTNet Epoch 2/100
Learning rate: 0.053144


  0%|          | 0/98 [00:00<?, ?it/s]

...

Train Loss: 3.0109, Train Acc: 26.42%
Val Loss: 3.2025, Val Acc: 23.32%
------------------------------------------------------------
LUTNet Epoch 6/100
Learning rate: 0.053144


  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

Train Loss: 3.0037, Train Acc: 26.51%
Val Loss: 3.2021, Val Acc: 23.18%
------------------------------------------------------------
LUTNet Epoch 7/100
Learning rate: 0.053144


  0%|          | 0/98 [00:00<?, ?it/s]

## Plot LUTNet Results


In [ ]:
# Plot LUTNet training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
ax1.plot(lut_train_losses, label='Train Loss', color='blue')
ax1.plot(lut_val_losses, label='Val Loss', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('LUTNet Training and Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy plot
ax2.plot(lut_train_accs, label='Train Acc', color='blue')
ax2.plot(lut_val_accs, label='Val Acc', color='red')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('LUTNet Training and Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

if len(lut_val_accs) > 0:
    print(f"LUTNet Best validation accuracy: {max(lut_val_accs):.2f}% at epoch {lut_val_accs.index(max(lut_val_accs))+1}")


In [161]:
filter_lut = Conv2DLUTLayer(
            input_shape=(30, 30),
            n_anchors_per_detector=8,
            detectors_shape=(6, 6),
            output_kernel_shape=(6, 6),
            receptive_field_shape=(5 * 6, 5 * 6),
            receptive_field_stride_shape=(2 * 6, 2 * 6),
            lut_receptive_field_shape=(6, 6),
            lut_receptive_field_stride_shape=(6, 6),
            weights_gradient_policy=g_policy,
            shared_context=shared_lut_ctx,
            synapse_meta=synapse_meta,
            summation_dtype=summation_dtype,
            random_seed=random_seed,
            _forward_group_size=32,
            _backward_group_size=32,
            _max_groups_in_growth_buffer=10**7,
            device=device
        )

In [162]:
filter_lut._output_shape

(6, 6)

In [44]:
filter_lut

Conv2DLUTLayer(input_shape=(56, 56), output_shape=(20, 20), detectors_shape=(4, 4), n_anchors_per_detector=8)

In [57]:
76 / 4

19.0

In [41]:
56 / 4

14.0

In [143]:
18 * 18

324